Import necessary library

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import urllib.request
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

Getting information about the events on the site from the api

In [2]:
def data_scrap(url: str):
    url = url
    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, "html.parser")

        h3_tags_title = soup.find_all("h3", class_="blog_post_title my-2")
        h3_tags_location = soup.find_all("div", class_="blog_post_title my-2")
        h3_tags_date = soup.find_all("div", class_="theater-date my-2")

        titles = []
        location = []
        date = []

        for title in h3_tags_title:
            if title.text.strip():
                titles.append(title.text.strip())

        df_title = pd.DataFrame({"Titles": titles})

        for loc in h3_tags_location:
            if loc.text.strip():
                location.append(loc.text.strip())

        df_location = pd.DataFrame({"Titles": location})

        for dt in h3_tags_date:
            if dt.text.strip():
                date.append(dt.text.strip())

        df_date = pd.DataFrame({"Titles": date})

    return df_title, df_date, df_location

In [3]:
# load events
df_title, df_date, df_location = data_scrap("https://www.6234.ir/")

In [4]:
df_title.isna()

,Titles
0,False
1,False
2,False
3,False
4,False
5,False
6,False
7,False
8,False
9,False


Getting information about the user buy and Interaction history on the site from the api

In [5]:
def user_buy_iteraction_from_api(buy_api: str, iter_api: str):
    buy_link = buy_api
    iter_link = iter_api
    urllib.request.urlretrieve(iter_link, "log.xlsx")
    iter_history = pd.read_excel("log.xlsx")
    urllib.request.urlretrieve(buy_link, "visitor.xlsx")
    buy_history = pd.read_excel("visitor.xlsx")

    return iter_history, buy_history

In [6]:
# load user buy and interaction from api

interaction, buy_history = user_buy_iteraction_from_api(
    "https://6234.ir/api/ticket?token=apiqazxcvbnm&ofDate=1402/08/20&toDate=1402/08/29",
    "https://6234.ir/api/log?token=apiqazxcvbnm&ofDate=1402/08/20&toDate=1402/12/29",
)

Data preparation

In [7]:
interaction.head()

,نام و نام خانوادگی,شماره موبایل,بازدید,زمان تعامل(تانیه),آی پی,تاریخ,تگ ها,توضیحات
0,ناشناس,NaN,صفحه اصلی,0.0,83.123.96.25,1402/12/28 21:17:53,NaN,NaN
1,ناشناس,NaN,صفحه اصلی,0.0,190.212.57.90,1402/12/28 21:17:42,NaN,NaN
2,ناشناس,NaN,صفحه اصلی,0.0,43.135.149.154,1402/12/28 19:33:12,NaN,NaN
3,ناشناس,NaN,صفحه اصلی,0.0,104.233.15.70,1402/12/28 19:11:35,NaN,NaN
4,ناشناس,NaN,صفحه اصلی,0.0,129.226.146.179,1402/12/28 18:45:03,NaN,NaN


Visiting is important here and we will fill the empty space

In [8]:
interaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30204 entries, 0 to 30203
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   نام و نام خانوادگی  30186 non-null  object 
 1   شماره موبایل        134 non-null    float64
 2   بازدید              30176 non-null  object 
 3   زمان تعامل(تانیه)   30204 non-null  float64
 4   آی پی               30204 non-null  object 
 5   تاریخ               30204 non-null  object 
 6   تگ ها               136 non-null    object 
 7   توضیحات             4644 non-null   object 
dtypes: float64(2), object(6)
memory usage: 1.8+ MB


In [9]:
interaction.isnull().sum()

نام و نام خانوادگی       18
شماره موبایل          30070
بازدید                   28
زمان تعامل(تانیه)         0
آی پی                     0
تاریخ                     0
تگ ها                 30068
توضیحات               25560
dtype: int64

In [10]:
interaction["بازدید"] = interaction["بازدید"].fillna("ffill")
interaction["نام و نام خانوادگی"] = interaction["نام و نام خانوادگی"].fillna("none")
interaction["شماره موبایل"] = interaction["شماره موبایل"].fillna("none")

In [11]:
# Drop صفحه اصلی from بازدید columns
interaction = interaction[interaction["بازدید"] != "صفحه اصلی"]

In [12]:
buy_history.head()

,شماره بلیط,نام و نام خانوادگی,شماره موبایل,ایمیل,تاریخ تولد,سن,جنسیت,استان سکونت,شهر سکونت,تحصیلات,...,صندلی,استان,شهر,سالن,تهیه کننده,صحنه گردان,رویداد,تاریخ سانس,ساعت سانس,تاریخ ثبت
0,1600,سفردوست,9101758962,NaN,1354/01/08,49,male,تهران,تهران,NaN,...,NaN,NaN,NaN,ترکیه,NaN,NaN,پارک امیرگان ( Test ),1 فروردین 1403,NaN,1402/08/20 18:12:26
1,1601,امیر نبی پور,9117803218,nabipour63@gmail.com,1363/01/01,40,male,تهران,تهران,کارشناسی ارشد,...,NaN,NaN,NaN,NaN,موسسه شکوه موسیقی ایرانیان,NaN,مسابقه آنلاین,1 فروردین 1403,20:00,1402/08/26 22:46:13


In [13]:
# merge all df
def merge_df(
    title_scrap: pd.DataFrame,
    location_scrap: pd.DataFrame,
    date_scrap: pd.DataFrame,
    itraction,
    buy_history,
):

    merge_df = pd.DataFrame(
        {
            "Titles": title_scrap["Titles"],
            "Location": location_scrap["Titles"],
            "Date": date_scrap["Titles"],
        }
    )

    merge_df = pd.concat(
        [merge_df["Titles"], itraction["بازدید"], buy_history["رویداد"]]
    ).reset_index()

    merge_df.columns = ["index", "Titles"]

    return merge_df

In [14]:
# use merge df func to merge all df

merged_df = merge_df(
    title_scrap=df_title,
    location_scrap=df_location,
    date_scrap=df_date,
    itraction=interaction,
    buy_history=buy_history,
)
merged_df.head()

,index,Titles
0,0,کنسرت تست ( بدون انتخاب صندلی )
1,1,کنسرت تست ( با انتخاب صندلی )
2,2,کنگره بین‌المللی جامعه دندانپزشکی ایران
3,3,دهمین همایش سالانه بانکداری الکترونیک
4,4,نمایشگاه و جشنواره هوش مصنوعی در گردشگری و تفر...


In [15]:
merged_df.isnull().sum()

index     0
Titles    0
dtype: int64

In [16]:
# use LabelEncoder for convert user(str) name to userId(int)
def iteraction_pre(df):
    df = df
    le = LabelEncoder()
    df["userId"] = le.fit_transform(df["نام و نام خانوادگی"])

    return df

In [17]:
df = iteraction_pre(interaction)

In [18]:
def list_to_string(row):
    return " ".join(row)

In [19]:
df_ohe = merged_df["Titles"].str.split(" ").reset_index().astype("str")

In [20]:
df_ohe["Titles"] = df_ohe["Titles"].apply(list_to_string)

In [21]:
le = LabelEncoder()
merged_df["ohe"] = le.fit_transform(df_ohe["Titles"])

In [22]:
def vectorized_text(df: pd.DataFrame):
    vectorized = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
    X = vectorized.fit_transform(merged_df["Titles"])

    feature_names = vectorized.get_feature_names_out()
    one_hot_df = pd.DataFrame(X.toarray(), columns=feature_names)

    dfs = pd.concat([df, one_hot_df], axis=1)
    dfs.drop(columns=["Titles"], inplace=True)

    return dfs

In [23]:
dfs = vectorized_text(df_title)

In [24]:
def creat_X(df):

    M = df["userId"].nunique()

    N = df["بازدید"].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))

    item_mapper = dict(zip(np.unique(df["بازدید"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))

    item_inv_mapper = dict(zip(list(range(N)), np.unique(df["بازدید"])))

    user_index = [user_mapper[i] for i in df["userId"]]

    item_indx = [item_mapper[i] for i in df["بازدید"]]

    X = csr_matrix((df["زمان تعامل(تانیه)"], (user_index, item_indx)), shape=(M, N))

    return X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper

In [25]:
X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = creat_X(df)

In [26]:
def find_similar_item(iter_name, X, item_mapper, item_inv_mapper, k, metrics="cosine"):

    X = X.T
    neighbours_ids = []

    iter_ind = item_mapper[iter_name]
    iter_vec = X[iter_ind]
    if isinstance(iter_vec, (np.ndarray)):
        iter_vec = iter_vec.reshape(1, -1)
    # use k+1 since kNN output includes the user recommender of interest
    knn = NearestNeighbors(n_neighbors=k + 1, algorithm="brute", metric=metrics)
    knn.fit(X)
    neighbours = knn.kneighbors(iter_vec, return_distance=False)
    for i in range(0, k):
        n = neighbours.item(i)
        neighbours_ids.append(item_inv_mapper[n])
    neighbours_ids.pop(0)
    return neighbours_ids

In [44]:
cosine_sim = cosine_similarity(dfs, dfs)
iter_idx = dict(zip(merged_df["Titles"].unique(), list(interaction.index)))
idx = iter_idx["کنسرت تست ( بدون انتخاب صندلی )"]
n_recommendations = 1
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1 : (n_recommendations + 1)]
similar_item = [i[0] for i in sim_scores]
recomended = merged_df["Titles"].iloc[similar_item]
recomended = recomended.to_list()
recomended

['کنسرت تست ( بدون انتخاب صندلی )']

In [42]:
idx

27888

In [43]:
cosine_sim.shape

(11400, 11400)

In [45]:
def cosine_similioraty(
    dfs: pd.DataFrame, merge_pd, itraction, idx: str, n_recommendations: int = 1
):
    cosine_sim = cosine_similarity(dfs, dfs)
    iter_idx = dict(zip(merged_df["Titles"].unique(), list(itraction.index)))
    idx = iter_idx[idx]
    n_recommendations = n_recommendations
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1 : (n_recommendations + 1)]
    similar_item = [i[0] for i in sim_scores]
    recomended = merge_pd["Titles"].iloc[similar_item]
    recomended = recomended.to_list()

    return recomended

In [46]:
cosine_similarity(dfs, dfs).shape

(11400, 11400)

In [47]:
users_phone = interaction["شماره موبایل"].unique()

In [49]:
user_iter = {}
for i in users_phone:
    user_it = (
        interaction[interaction["شماره موبایل"] == i][["زمان تعامل(تانیه)", "بازدید"]]
        .max()
        .reset_index()
        .T
    )
    user_it.columns = ["زمان تعامل(تانیه)", "بازدید"]
    user_it.drop(index="index", inplace=True)
    user_it["بازدید"]
    idx = user_it["بازدید"].to_list()[0]
    names = i
    iters = cosine_similioraty(dfs, merged_df, interaction, idx=idx)
    user_dict = {names: iters}
    user_iter.update(user_dict)
user_iter

{'none': ['تور دبی'],
 9101022210.0: ['تور قشم'],
 9145236873.0: ['کنسرت نمایش کلنل'],
 9101758962.0: ['پارک امیرگان ( Test )'],
 9122070929.0: ['تور پوکت'],
 9117803218.0: ['دهمین همایش سالانه بانکداری الکترونیک']}